# Carga y Limpieza de Datos

In [1]:
# Importo paquetes

import numpy as np
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
%matplotlib inline

from scipy import sparse
from sklearn.preprocessing import StandardScaler

In [2]:
X_train = pd.read_csv("./Data/X_train_LDA.csv")
X_test = pd.read_csv("./Data/X_test_LDA.csv")
y_train = pd.read_csv("./Data/y_train.csv", header=None)
y_test = pd.read_csv("./Data/y_test.csv", header=None)

In [3]:
X_train.size / 1024 /1024 /1024

0.2654140815138817

In [4]:
y_train = y_train[0]
y_test = y_test[0]

In [5]:
se = StandardScaler()
X_train = se.fit_transform(X_train)
X_test = se.transform(X_test)

In [5]:
X_train.size / 1024 /1024 /1024

0.2654140815138817

# Modelado

In [6]:
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, BaggingClassifier, AdaBoostClassifier
from sklearn.metrics import confusion_matrix, recall_score, accuracy_score, precision_score, roc_curve, roc_auc_score, auc
from xgboost.sklearn import XGBClassifier
import lightgbm as lgb
import scipy.stats as st
from sklearn.externals import joblib

## 1. Naive Bayes + Regresión Logística

In [7]:
models = [BernoulliNB(), LogisticRegression()]

In [8]:
params = [{},
          {'C': np.random.normal(1, 0.2, 10).astype(float) }]

In [9]:
grids = []
i = 0
for i in range(len(models)):
    gs = RandomizedSearchCV(models[i], params[i], n_iter = 10, scoring='roc_auc', verbose= 10, n_jobs=-1, cv=3)
    print (gs)
    fit = gs.fit(X_train, y_train)
    grids.append(fit)

RandomizedSearchCV(cv=3, error_score='raise-deprecating',
          estimator=BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True),
          fit_params=None, iid='warn', n_iter=10, n_jobs=-1,
          param_distributions={}, pre_dispatch='2*n_jobs',
          random_state=None, refit=True, return_train_score='warn',
          scoring='roc_auc', verbose=10)
Fitting 3 folds for each of 1 candidates, totalling 3 fits


C:\Users\juan\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:271: UserWarning: The total space of parameters 1 is smaller than n_iter=10. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:  2.9min finished
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:  2.9min remaining:    0.0s


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
          estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False),
          fit_params=None, iid='warn', n_iter=10, n_jobs=-1,
          param_distributions={'C': array([0.91776, 0.85676, 0.95503, 0.9766 , 1.20022, 0.6465 , 1.13456,
       1.06029, 0.98411, 1.23645])},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring='roc_auc', verbose=10)
Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done  16 out of  19 | elapsed:  1.6min remaining:   17.5s


MemoryError: 

Falla de memoria: irónicamente al reducir la dimensionalidad de +34k a 200 aumenta el tamaño del dataset por tener que pasar de una matriz sparse a una densa. Tengo entonces pérdida de información y la ganancia potencial por reducción de la maldición de la dimensionalidad no parece sustancial dado el gran numero de observaciones vs features (1,4M vs. 34k)

Abandonamos este enfoque